In [ ]:
import os
import requests

In [ ]:
from bs4 import BeautifulSoup
from settings import (
    DOWNLOAD_URL, 
    GVA_DOWNLOAD_AGAIN,
    GVA_HOST,
    REGISTER_FILENAME,
)

In [ ]:
LOCAL_REGISTER_FILENAME = f'../data/{REGISTER_FILENAME}'

In [ ]:
download_again = (not os.path.exists(LOCAL_REGISTER_FILENAME)) or (GVA_DOWNLOAD_AGAIN)

In [ ]:
if download_again:
    web_response = requests.get(f'{GVA_HOST}/{DOWNLOAD_URL}')

    webcontent = web_response.text
    soup = BeautifulSoup(webcontent, 'html.parser')

    register_url = [ reg.get('href') for reg in soup.select('a') if REGISTER_FILENAME in reg.get('href','')]

    register_link = f'{GVA_HOST}{register_url[0]}'
    
    with requests.get(register_link, stream=True) as register_response:
        # register_response.raise_for_status()
        with open(LOCAL_REGISTER_FILENAME, 'wb') as lrf:
            for chunk in register_response.iter_content(chunk_size=8192):
                lrf.write(chunk)

In [ ]:
## https://www.nosolosig.com/articulos/1191-asi-hice-el-mapa-de-los-codigos-postales-de-espana-con-sig-y-datos-abiertos